In [101]:
from tools.ground_removal import *
from tools.raycasting import *
import numpy as np
from sklearn.cluster import DBSCAN
from mayavi import mlab
from tools.visualization import *
from scipy.spatial import KDTree
import yaml
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
gr = Ground_removal('01')
first_frame = 1
second_frame = first_frame + 2
pts, intens = gr.get_synchronized_frames_without_ground(first_frame,1)
pts2, intens2 = gr.get_synchronized_frames_without_ground(second_frame,1)

In [164]:
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], color=(0,1,0), mode='point')
mlab.points3d(pts2[:,0], pts2[:,1], pts2[:,2], color=(0,0,1), mode='point')
mlab.show()

In [165]:
clustering = DBSCAN(eps=0.6, min_samples=10).fit(pts[:,:3])
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], clustering.labels_, mode='point')
mlab.show()

In [317]:
with open('raycast_config.yaml', 'r') as f:
    config = yaml.safe_load(f)
#figure = mlab.figure()
rp = RaycastPredictor(config['CAR'], gr)
mask = rp.predict(0)

In [122]:
# Evaluation
from tools.evaluation import evaluate_kitti
with open('raycast_config.yaml', 'r') as f:
    raycast_config = yaml.safe_load(f)
with open('evaluate_cofig.yaml', 'r') as f:
    evaluation_config = yaml.safe_load(f)

In [84]:
evaluate_kitti(evaluation_config, RaycastPredictor, raycast_config)

class PEDESTRIAN
    sequence: 01
        frame: 000000 -> precision 100%, recall 100%, iou 100%
        frame: 000001 -> precision 100%, recall 100%, iou 100%
        frame: 000002 -> precision 100%, recall 100%, iou 100%
        frame: 000003 -> precision 100%, recall 100%, iou 100%
        frame: 000004 -> precision 100%, recall 100%, iou 100%
        frame: 000005 -> precision 100%, recall 100%, iou 100%
        frame: 000006 -> precision 100%, recall 100%, iou 100%
        frame: 000007 -> precision 100%, recall 100%, iou 100%
        frame: 000390 -> precision 0%, recall 100%, iou 0.0%
        frame: 000391 -> precision 0%, recall 100%, iou 0.0%
        frame: 000392 -> precision 0%, recall 100%, iou 0.0%
        frame: 000393 -> precision 0%, recall 100%, iou 0.0%
        frame: 000394 -> precision 0%, recall 100%, iou 0.0%
        frame: 000395 -> precision 0%, recall 100%, iou 0.0%
        frame: 000396 -> precision 0%, recall 100%, iou 0.0%
    average sequence precision 53.3

[79.95, 71.65, 51.61]

In [123]:
# closer investigation
gr = Ground_removal('05')
figure = mlab.figure()
rp = RaycastPredictor(raycast_config['PEDESTRIAN'], gr, figure, verbose=True)
#rp.predict(580); pts, intens = gr.get_frame(585)
#mlab.points3d(pts[:,0], pts[:,1], pts[:,2], intens, figure=figure, mode='point')
plot_prediction_pedestrians(gr, rp, 606, figure)
mlab.show()

skipping cluster 0 because of its length 7.784990156476539
skipping cluster 4 because of its length 6.493863560320619
skipping cluster 8 because of its length 6.945311537634723
skipping cluster 9 because of its length 11.804226568581697
skipping cluster 10 because of its length 3.5351801798841223
skipping cluster 23 because of its length 2.0365542066358557
skipping cluster 30 because of its length 2.053924154652549
skipping cluster 32 because of its length 2.3904074577645247
skipping cluster 43 because of its length 2.4371958442987705
skipping cluster 56 because of its length 7.7369150279363055
skipping cluster 59 because of its length 4.048820520411937
ray casting to cluster 6 to future : Number of checks for raycast is 56, ray has not reached the target
ray casting to cluster 37 to future : Number of checks for raycast is 41, ray has not reached the target
ray casting to cluster 52 to future : Number of checks for raycast is 77, ray has not reached the target
ray casting to cluster 5

### TODO figure out why cluster 13 is only raycasted to past and not future

In [117]:
pts, intens = gr.get_frame_without_ground(606)
clustering = DBSCAN(eps=raycast_config['PEDESTRIAN']['EPS'],
                    min_samples=raycast_config['PEDESTRIAN']['MIN_SAMPLES']).fit(pts[:,:3])
mask = clustering.labels_ == 13
mlab.points3d(pts[:,0][mask], pts[:,1][mask], pts[:,2][mask], color=(1,0,0), mode='point')
mlab.points3d(pts[:,0][~mask], pts[:,1][~mask], pts[:,2][~mask], color=(1,1,1), mode='point')
mlab.show()

In [125]:
pts, intens = gr.get_frame_without_ground(601)
pts2, intens = gr.get_frame_without_ground(606)
pts3, _ = gr.get_frame_without_ground(611)
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], color=(1,0,0), mode='point')
mlab.points3d(pts2[:,0], pts2[:,1], pts2[:,2], color=(0,1,0), mode='point')
mlab.points3d(pts3[:,0], pts3[:,1], pts3[:,2], color=(0,0,1), mode='point')
mlab.show()

In [119]:
pts, intens = gr.get_synchronized_frames_without_ground(601,5)
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], pts[:,3], mode='point')
mlab.show()

In [25]:
cluster_pts = pts[mask]
bb_x_min =  cluster_pts[:,0].min()
bb_x_max =  cluster_pts[:,0].max()
bb_y_min =  cluster_pts[:,1].min()
bb_y_max =  cluster_pts[:,1].max()
bb_z_min =  cluster_pts[:,2].min()
bb_z_max =  cluster_pts[:,2].max()

width = min(abs(bb_x_max - bb_x_min), abs(bb_y_max - bb_y_min))
length = max(abs(bb_x_max - bb_x_min), abs(bb_y_max - bb_y_min))
height = abs(bb_z_max - bb_z_min)
print(width,length, height)

3.8669458653616857 4.55708538003114 1.7237112914907087


### grid search parameters tuning

In [423]:
from tools.evaluation import grid_search
with open('raycast_config.yaml', 'r') as f:
    raycast_config = yaml.safe_load(f)
with open('evaluate_cofig.yaml', 'r') as f:
    evaluation_config = yaml.safe_load(f)
    
params = {'EPS': [0.4,0.5], 'RADIUS_RAYCAST': [0.2,0.3]}
grid_search(evaluation_config, RaycastPredictor, raycast_config, params)

{'EPS': 0.4, 'MIN_SAMPLES': 10, 'RADIUS_EMPTY_SPACE_CHECK': 1, 'RADIUS_RAYCAST': 0.3, 'MAX_WIDTH': 3, 'MAX_LENGTH': 6, 'MAX_HEIGTH': 2, 'MIN_LENGTH': 1, 'NUM_OF_FRAMES_IN_FUTURE': 2, 'EGO_DELETE_RADIUS': 2.5}
Error, path ../../semantic_kitti_data/sequences/05/velodyne/000578.bin does not exists
Error, path ../../semantic_kitti_data/sequences/05/velodyne/000579.bin does not exists
[95.68, 58.57, 57.93]
{'EPS': 0.5, 'MIN_SAMPLES': 10, 'RADIUS_EMPTY_SPACE_CHECK': 1, 'RADIUS_RAYCAST': 0.3, 'MAX_WIDTH': 3, 'MAX_LENGTH': 6, 'MAX_HEIGTH': 2, 'MIN_LENGTH': 1, 'NUM_OF_FRAMES_IN_FUTURE': 2, 'EGO_DELETE_RADIUS': 2.5}
Error, path ../../semantic_kitti_data/sequences/05/velodyne/000578.bin does not exists
Error, path ../../semantic_kitti_data/sequences/05/velodyne/000579.bin does not exists
[94.29, 60.92, 60.92]
{"{'EPS': 0.4, 'MIN_SAMPLES': 10, 'RADIUS_EMPTY_SPACE_CHECK': 1, 'RADIUS_RAYCAST': 0.3, 'MAX_WIDTH': 3, 'MAX_LENGTH': 6, 'MAX_HEIGTH': 2, 'MIN_LENGTH': 1, 'NUM_OF_FRAMES_IN_FUTURE': 2, 'EGO_